In [9]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

# ========== 1. 路径配置 ==========
BTC_EXCEL_PATH = r"C:\Users\33759\Desktop\btc数据挖掘\初始数据\btc_1d_data_2018_to_2025.xlsx"
FNG_EXCEL_PATH = r"C:\Users\33759\Desktop\btc数据挖掘\初始数据\fear_greed_index.xlsx"
SAVE_DIR = r"C:\Users\33759\Desktop\btc数据挖掘\修改后数据"

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    print(f"✅ 已创建文件夹：{SAVE_DIR}")

# ========== 2. 处理BTC数据（核心：日期仅保留年-月-日） ==========
btc_df = pd.read_excel(BTC_EXCEL_PATH)

# 解析UTC日期 + 仅保留年-月-日（去除时分秒）
btc_df['Open time'] = pd.to_datetime(
    btc_df['Open time'],
    format='%Y-%m-%d %H:%M:%S.%f UTC',
    utc=True
)
# 关键：提取日期部分（仅年-月-日）
btc_df['日期'] = btc_df['Open time'].dt.date
btc_df['日期'] = pd.to_datetime(btc_df['日期'])

# 计算涨跌幅（百分比）
btc_df['涨跌幅（百分比）'] = (btc_df['Close'].pct_change() * 100).round(2)
# 调整格式 + 重命名列
btc_df['收盘价'] = btc_df['Close'].astype(int)
btc_df.rename(columns={'Volume': '成交量'}, inplace=True)

# 保留核心列
btc_clean_df = btc_df[['日期', '收盘价', '成交量', '涨跌幅（百分比）']].copy()
btc_clean_df = btc_clean_df.drop_duplicates(subset='日期', keep='first')
btc_clean_df = btc_clean_df.dropna(subset=['收盘价', '成交量', '涨跌幅（百分比）'])

print(f"✅ BTC数据处理完成：{len(btc_clean_df)}条有效数据")

# ========== 3. 处理FNG情绪数据 ==========
fng_df = pd.read_excel(FNG_EXCEL_PATH)

# 解析日-月-年日期 + 仅保留年-月-日
fng_df['日期'] = pd.to_datetime(fng_df['date'], format='%d-%m-%Y').dt.date
fng_df['日期'] = pd.to_datetime(fng_df['日期'])

# 重命名列
fng_df.rename(columns={
    'fng_value': '情绪值',
    'fng_classification': '情绪分类'
}, inplace=True)
fng_clean_df = fng_df[['日期', '情绪值', '情绪分类']].copy()
fng_clean_df = fng_clean_df.drop_duplicates(subset='日期', keep='first')
fng_clean_df = fng_clean_df.dropna(subset=['情绪值', '情绪分类'])

print(f"✅ FNG数据处理完成：{len(fng_clean_df)}条有效数据")

# ========== 4. 合并数据 ==========
merge_df = pd.merge(btc_clean_df, fng_clean_df, on='日期', how='inner')
merge_df['次日涨跌幅（百分比）'] = merge_df['涨跌幅（百分比）'].shift(-1).round(2)
merge_df = merge_df.dropna(subset=['次日涨跌幅（百分比）'])

print(f"✅ 数据合并完成：{len(merge_df)}条最终数据")

# ========== 5. 保存为Excel ==========
# 保存时将日期转为字符串格式
merge_df['日期'] = merge_df['日期'].dt.strftime('%Y-%m-%d')
save_path = os.path.join(SAVE_DIR, "BTC_FNG_最终合并数据.xlsx")
merge_df.to_excel(save_path, index=False, engine='openpyxl')
print(f"✅ 最终数据已保存至：{save_path}")

✅ BTC数据处理完成：2920条有效数据
✅ FNG数据处理完成：2886条有效数据
✅ 数据合并完成：2885条最终数据
✅ 最终数据已保存至：C:\Users\33759\Desktop\btc数据挖掘\修改后数据\BTC_FNG_最终合并数据.xlsx
